In [1]:
import pandas as pd
import numpy as np

### XGBOOST

In [2]:
df_train = pd.read_csv('../data/train_con_features.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('../data/test_con_features.csv', index_col='Unnamed: 0')
display(df_train.head())

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,piscina,escuelascercanas,centroscomercialescercanos,precio,cantidad_amenities,cantidad_servicios,relacion_metros,metros_por_habitacion,metros_por_bano,metros_por_amenities
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116114,2.0,1.0,...,0.0,0.0,0.0,2273000.0,0.0,0.0,1.000000,40.000000,40.0,0.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,...,0.0,1.0,1.0,3600000.0,0.0,2.0,1.488889,89.333333,134.0,0.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,...,0.0,0.0,0.0,1200000.0,0.0,0.0,0.867470,48.000000,72.0,0.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,...,0.0,1.0,1.0,650000.0,0.0,2.0,0.940299,31.500000,63.0,0.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,...,0.0,0.0,0.0,1150000.0,0.0,0.0,1.000000,47.500000,95.0,0.0


In [3]:
columnas_numericas = ['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 
                      'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios']

columnas_numericas_y_precio = ['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 
                               'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios']

#'metros_por_bano', 'metros_por_habitacion'
#'relacion_metros', 'metros_por_amenities'

columnas_numericas_y_precio.append('precio')

In [4]:
X = df_train[columnas_numericas_y_precio].drop(columns=['precio']).values
y = df_train['precio'].values

In [5]:
# Métrica de evaluación
def RMSLE(actual, pred):
    return (np.mean((np.log(actual + 1) - np.log(pred + 1)) ** 2)) **.5

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(216000, 10) (216000,)
(24000, 10) (24000,)


In [7]:
import xgboost as xgb
XGBoost = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05, 
                           max_depth = 5, alpha = 10, n_estimators = 100)

XGBoost.fit(X_train, y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.3, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=None, subsample=1, verbosity=1)

In [8]:
XGBoost_pred = XGBoost.predict(X_test)

In [9]:
XGBoost_rmsle = RMSLE(y_test, XGBoost_pred)
XGBoost_rmsle_train = RMSLE(y_train, XGBoost.predict(X_train))
print(f"RMSLE XGBoost (train): {XGBoost_rmsle_train:.5f}")
print(f"RMSLE XGBoost: {XGBoost_rmsle:.5f}")
print("------------------------------")

RMSLE XGBoost (train): 0.74249
RMSLE XGBoost: 0.74895
------------------------------


In [10]:
df_train_sumbit = df_train[columnas_numericas_y_precio]
df_test_submit = df_test[columnas_numericas]
XGBoost_pred_sumbit = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05, 
                           max_depth = 5, alpha = 10, n_estimators = 100)\
                        .fit(df_train_sumbit[columnas_numericas], df_train_sumbit['precio'])\
                        .predict(df_test_submit[columnas_numericas])

C:\Users\maxi_\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\maxi_\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [11]:
resultado_submit = pd.DataFrame(index=df_test.index)
resultado_submit['id'] = df_test['id']
resultado_submit['target'] = XGBoost_pred_sumbit
display(resultado_submit.head())
resultado_submit.to_csv('../data/sumbitXGBoost.csv',index=False)

,id,target
0,4941,3574004.750
1,51775,1523771.750
2,115253,1346988.750
3,299321,2083434.625
4,173570,950742.375


#### Score en Kaggle (features => score):
 **1.** *['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios'] =>* ***1310457.48052*** con objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3
 
 **2.** *['antiguedad', 'habitaciones', 'garages', 'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'cantidad_amenities', 'cantidad_servicios'] =>* ***1332492.86254*** con objective='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.05, max_depth = 5, alpha = 10, n_estimators = 100